## Create the proteome NSAF of all proteins not connected to the tissue 

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import sqlite3
import mysql.connector

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas2', auth_plugin='mysql_native_password')
mycursor = conn.cursor()

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")


connection succesfull


In [3]:
assaysql = "SELECT assay_id, peptide_id, quantification FROM peptide_to_assay"
assayData = pd.read_sql_query(assaysql, conn)
assayData.head()

,assay_id,peptide_id,quantification
0,13988,62701207,208.0
1,13989,62701207,32.0
2,13990,62701207,30.0
3,13991,62701207,275.0
4,13992,62701207,36.0


Get all peptide to protein relations

In [4]:
pepsql = "SELECT peptide_to_protein.peptide_id, peptide_to_protein.uniprot_id FROM peptide_to_protein"
pepData = pd.read_sql_query(pepsql, conn)
pepData.head()

,peptide_id,uniprot_id
0,62815839,A0A024RBG1
1,63242375,A0A024RBG1
2,71023901,A0A024RBG1
3,63327481,A0A075B6H7
4,64466299,A0A075B6H7


Get sequence length for all proteins in database

In [5]:
seqsql = "SELECT uniprot_id, length FROM protein WHERE length IS NOT NULL"
seqData = pd.read_sql_query(seqsql, conn)
#seqData

In [6]:
seqData['length'] = pd.to_numeric(seqData['length'], errors='coerce')

In [7]:
seqData.head()

,uniprot_id,length
0,A0A024RBG1,181.0
1,A0A075B6H7,116.0
2,A0A075B6H8,117.0
3,A0A075B6H9,119.0
4,A0A075B6I0,122.0


Select proteotypic peptides: peptides which have only 1 peptide-protein relation

In [8]:
proteotypicData = pepData.groupby("peptide_id").filter(lambda x: len(x) == 1)
#proteotypicData

Protein inference: Only keep proteins which have at least 3 proteotypic peptides

In [9]:
proteins = proteotypicData.groupby("uniprot_id").filter(lambda x: len(x) > 2)
proteins.shape

(248293, 2)

In [10]:
non_human_proteins = ['TRYP_PIG', 'TRY2_BOVIN','TRY1_BOVIN','SSPA_STAAU','SRPP_HEVBR','REF_HEVBR', 'ADH1_YEAST', 'ALBU_BOVIN', 'CAS1_BOVIN', 'CAS2_BOVIN', 'CASK_BOVIN', 'CASB_BOVIN', 'OVAL_CHICK', 'ALDOA_RABIT', 'BGAL_ECOLI', 'CAH2_BOVIN', 'CTRA_BOVIN', 'CTRB_BOVIN', 'CYC_HORSE', 'DHE3_BOVIN', 'GAG_SCVLA', 'GFP_AEQVI', 'K1C15_SHEEP', 'K1M1_SHEEP', 'K2M2_SHEEP', 'K2M3_SHEEP', 'KRA3A_SHEEP', 'KRA3_SHEEP', 'KRA61_SHEEP', 'LALBA_BOVIN', 'LYSC_CHICK', 'LYSC_LYSEN', 'MYG_HORSE', 'K1M2_SHEEP', 'K2M1_SHEEP']

In [11]:
proteins = proteins[~proteins['uniprot_id'].isin(non_human_proteins)]

Merge assays containing spectral counts and proteins

In [12]:
protData = pd.merge(assayData, proteins, on = 'peptide_id').sort_values(['assay_id','uniprot_id'])
protData.head(50)
#protData

,assay_id,peptide_id,quantification,uniprot_id
193943,13988,62702249,2.0,A2RRP1
331215,13988,62703065,1.0,A5YM72
377865,13988,62703308,1.0,O00483
632303,13988,62704266,1.0,O00483
61303,13988,62701532,1.0,O14495
530855,13988,62703879,2.0,O14594
753765,13988,62704787,1.0,O14594
773938,13988,62704874,1.0,O14594
55284,13988,62701495,1.0,O14926
2214,13988,62701226,2.0,O15020


In [13]:
del protData['peptide_id']

Split data per assay

In [14]:
assays = protData['assay_id'].unique()
DataFrameDict = {elem : pd.DataFrame for elem in assays}
for key in DataFrameDict.keys():
    DataFrameDict[key] = protData[:][protData['assay_id'] == key]

Calculate NSAF score for each protein per assay

In [15]:
DataFrameDict2 = DataFrameDict.copy()

In [16]:
for key in DataFrameDict2.keys():
    sumSaf = 0
    assay = DataFrameDict2[key]
    assay.pop('assay_id')
    # calculate sum of spectral counts for each protein
    grouped = DataFrameDict2[key].groupby('uniprot_id').sum().reset_index()
    seqAddedDF = pd.merge(grouped, seqData, on = 'uniprot_id')
    seqAddedDF.insert(loc = 2, column = 'SAF', value = 0)
    seqAddedDF.insert(loc = 3, column = 'NSAF', value = 0)

    # calculate SAF score for each protein by dividing sum of spectral counts by protein length
    for index, row in seqAddedDF.iterrows():
        saf = row['quantification']/row['length']
        seqAddedDF.loc[index, 'SAF']  = saf
        # calculate sum of SAF scores in assay
        sumSaf = sumSaf + saf

    # Calculate NSAF score by normalizing each SAF score
    seqAddedDF['NSAF'] = seqAddedDF['SAF']/ sumSaf

    del seqAddedDF['length']
    del seqAddedDF['quantification']
    del seqAddedDF['SAF']
    seqAddedDF.insert(loc = 0, column = 'assay_id', value = key)
    DataFrameDict2[key] = seqAddedDF


ValueError: cannot insert assay_id, already exists

In [ ]:
proteinData = pd.DataFrame()

for key in DataFrameDict2.keys():
    proteinData = proteinData.append(DataFrameDict2[key])


In [ ]:
proteinData.head()

,assay_id,uniprot_id,NSAF
0,13988,A2RRP1,0.000259
1,13988,A5YM72,0.000372
2,13988,O00483,0.007590
3,13988,O14495,0.000988
4,13988,O14594,0.000931


In [ ]:
# write NSAF proteome to file
proteinData.to_csv(path_or_buf = '../Atlas_and_parser/Atlas_output/proteome_nsaf_update0806.csv', index = False)

In [3]:
df = pd.read_csv( '../Atlas_and_parser/Atlas_output/proteome_nsaf_update0806.csv')
df.head()

,assay_id,uniprot_id,NSAF
0,13988,A2RRP1,0.000259
1,13988,A5YM72,0.000372
2,13988,O00483,0.007590
3,13988,O14495,0.000988
4,13988,O14594,0.000931
